This notebook reproduces tests in Ripperda+ 2018: https://ui.adsabs.harvard.edu/abs/2018ApJS..235...21R/abstract

More tests/methods will be included with time. Using CGS units instead of MKS of that paper.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import sys
import os

In [2]:
#parameters in cgs units
m = 9.1e-27 #electron's mass in CGS units (gram)
e = 5.e-10 #electron's charge in CGS units (esu) 
qbym = -e/m #electron's charge to mass ratio 
c = 3.e10 #speed of light

prob = 'EcrossB'
method = 'Boris'

if (prob == 'UniE'):
    qbym = 1.
    c = 1.

if (prob == 'UniB'):
    qbym = 1.
    c = 1.
    
if (prob == 'ForceFree'):
    qbym = 1.
    c = 1.
    
if (prob == 'EcrossB'):
    qbym = 1.
    c = 1.    

In [3]:
def cal_E(x, prob = 'UniE'):
    if (prob == 'UniE'):
        return np.array([1.0, 0.0, 0.0])
    if (prob == 'UniB'):
        return np.array([0.0, 0.0, 0.0])
    if (prob == 'ForceFree'):
        return np.array([-(1-0.5/1.e12), 0.0, 0.0])
    if (prob == 'EcrossB'):
        return np.array([1. - 5.e-5, 0.0, 0.0])

def cal_B(x, prob = 'UniE'): 
    if (prob == 'UniE'):
        return np.array([0.0, 0.0, 0.0])
    if (prob == 'UniB'):
        return np.array([0.0, 0.0, 1.e6])
    if (prob == 'ForceFree'):
        return np.array([0.0, 0.0, 1.0])
    if (prob == 'EcrossB'):
        return np.array([0., 0., 1.])

In [4]:
#initial conditions
if (prob == 'UniE'):
    xp = np.array([0., 0., 0.])
    up = np.array([0., 0., 0.]) #4-velocity
if (prob == 'UniB'):
    xp = np.array([0., 0., 0.])
    up = np.array([0., -1.e6*(1-0.5*1e-12), 0.]) #4-velocity
if (prob == 'ForceFree'):
    xp = np.array([0., 0., 0.])
    up = np.array([0., 1.e6*(1-0.5*1e-12), 0.]) #4-velocity
if (prob == 'EcrossB'):
    xp = np.array([0., 0., 0.])
    up = np.array([0., 0., 0.]) #4-velocity

In [5]:
if (prob == 'UniE'):
    tend = 1.e9
    dt = 1.e3
if (prob == 'UniB'):
    w_c0 = 1./np.abs(qbym)
    tend = 100*2.0*np.pi/w_c0 #evolve for 100 Larmor orbits
    dt = 0.01*2*np.pi/w_c0 #1/100th of a cyclotron orbit
if (prob == 'ForceFree'):
    tend = 1.e5
    dt = 1.0
if (prob == 'EcrossB'):
    tend = 2.*np.pi*1.e5
    dt = 0.5
dt0 = dt

In [6]:
time = 0
data_t = []
data_t.append(np.append(xp,up)) #appends to the list; list to hold 6 coordinates at each time

In [7]:
def update_xpup(dt, xp, up, qbym, c, method = 'Boris'): #synchronised leapfrog; Eqs. 6-8 in Ripperda+ 2018
    gamma = np.sqrt( 1 + np.inner(up, up)/(c*c) )
    xp += 0.5*dt*up/gamma #x^{n+1/2}
    Ep = cal_E(xp, prob)
    Bp = cal_B(xp, prob)
    if (method == 'Boris'):
        up += 0.5*dt*qbym*Ep #uminus
        gamma = np.sqrt( 1 + np.inner(up, up)/(c*c) )
        t = Bp*0.5*dt*qbym/(c*gamma)
        s = 2*t/(1 + np.inner(t, t))
        up += np.cross((up + np.cross(up, t)), s) #uplus
        up += 0.5*dt*qbym*Ep #u^{n+1}
    if (method == 'Vay'):
        up += 0.5*dt*qbym*(Ep + np.cross(up, Bp)/(c*gamma) ) #u^{n+1/2}
        up += 0.5*dt*qbym*Ep #uprime
        gamma = np.sqrt( 1 + np.inner(up, up)/(c*c) ) #gamma^\prime
        t = 0.5*Bp*qbym*dt/c #tau
        ustar = np.inner(up, t)/c
        tau_sqr = np.inner(t, t)
        sig = gamma*gamma - tau_sqr
        gamma = np.sqrt( 0.5*( sig + np.sqrt(sig*sig + 4*(tau_sqr + ustar*ustar) ) ) ) #gamma^{n+1}
        t /= gamma #t
        s = 1/( 1 + np.inner(t, t) ) #s
        up = s*( up + np.inner(up, t)*t + np.cross(up, t) ) #u^{n+1}
        #sys.exit()
    if (method == 'HC'):
        up += 0.5*dt*qbym*Ep #uminus
        gamma = np.sqrt( 1 + np.inner(up, up)/(c*c) ) #gamma^-
        t = 0.5*Bp*qbym*dt/c #tau
        ustar = np.inner(up, t)/c
        tau_sqr = np.inner(t, t)
        sig = gamma*gamma - tau_sqr
        gamma = np.sqrt( 0.5*( sig + np.sqrt(sig*sig + 4*(tau_sqr + ustar*ustar) ) ) ) #gamma^+
        t /= gamma #t
        s = 1/( 1 + np.inner(t, t) ) #s
        up = s*(up + np.inner(up, t)*t + np.cross(up, t) ) #u^+
        up += 0.5*dt*qbym*Ep + np.cross(up, t)
        
    gamma = np.sqrt( 1 + np.inner(up, up)/(c*c) ) #gamma^{n+1}
    xp += 0.5*dt*up/gamma
    return xp, up

In [ ]:
while (time < tend):
    dt = min(dt, tend-time)
    time += dt
    xp, up = update_xpup(dt, np.copy(xp), np.copy(up), qbym, c, method = method)
    if (time != tend):
        data_t.append(np.append(xp, up))

In [ ]:
data_t = np.array(data_t)
t = np.linspace(0, tend, np.size(data_t[:,0]))
if (prob == 'UniE'):
    #plt.plot(t, data_t[:,0])
    plt.plot(t, np.sqrt(1. + data_t[:,3]*data_t[:,3]/(c*c)) )
if (prob == 'UniB'):
    plt.plot(data_t[:,0], data_t[:,1],'.', markersize=0.1)
    plt.axis('equal')
if (prob == 'ForceFree'):
    plt.plot(t, data_t[:,0])
    #plt.plot(t, data_t[:,3])
if (prob == 'EcrossB'):
    plt.plot(data_t[:,0], data_t[:,1],'.', markersize=0.1)
plt.grid()